In [1]:
import logging
import sys
from pathlib import Path

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s")

In [2]:
from tplexity.retriever.retriever_service import RetrieverService

service = RetrieverService()

/home/nlp1_user01/.cache/pypoetry/virtualenvs/tplexity-AFAH5PUx-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-11-16 23:58:04,625 - tplexity.retriever.retriever_service - INFO - 🔄 [retriever_service] Инициализация гибридного поисковика
2025-11-16 23:58:04,626 - tplexity.retriever.vector_search - INFO - 🔄 [retriever][vector_search] Инициализация клиента Qdrant
2025-11-16 23:58:04,845 - httpx - INFO - HTTP Request: GET https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333 "HTTP/1.1 200 OK"
2025-11-16 23:58:04,846 - tplexity.retriever.vector_search - INFO - ✅ [retriever][vector_search] Клиент Qdrant инициализирован: 8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333
2025-11-16 23:58:04,944 - tplexity.retriever.utils - INFO - ✅ [device] G

In [5]:
count_result = await service.vector_search.client.count(collection_name=service.vector_search.collection_name)
print(f"Количество документов в коллекции '{service.vector_search.collection_name}': {count_result.count}")

2025-11-16 23:59:45,135 - httpx - INFO - HTTP Request: POST https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections/documents/points/count "HTTP/1.1 200 OK"


Количество документов в коллекции 'documents': 1000


In [ ]:
documents = [
    "Банковский вклад - это денежные средства, размещенные в банке на определенный срок под проценты. Вклады бывают срочные и до востребования.",
    "Кредит - это заем денежных средств, предоставляемый банком клиенту на определенных условиях. Кредиты бывают потребительские, ипотечные и автокредиты.",
    "Для открытия счета в банке необходимо предоставить паспорт, заполнить заявление и внести первоначальный взнос. Счет можно открыть онлайн или в отделении банка.",
    "Дебетовая карта позволяет расплачиваться собственными средствами, хранящимися на счете. Кредитная карта позволяет тратить заемные средства банка.",
    "Ипотечный кредит выдается на покупку недвижимости. Обычно требуется первоначальный взнос от 10% до 30% от стоимости недвижимости.",
]

metadatas = [
    {"type": "deposit", "category": "banking"},
    {"type": "credit", "category": "banking"},
    {"type": "account", "category": "banking"},
    {"type": "card", "category": "banking"},
    {"type": "mortgage", "category": "credit"},
]

# await service.add_documents(documents, metadatas=metadatas)
# print(f"Добавлено документов: {len(documents)}")

In [6]:
results = await service.search("Что такое банковский вклад?", top_k=10, top_n=5, use_rerank=False)
print(f"Найдено результатов: {len(results)}")
for doc_id, score, text, metadata in results:
    print(f"Doc ID: {doc_id}, Score: {score:.3f}")
    print(f"Text: {text[:100]}...")
    print(f"Metadata: {metadata}")
    print()

2025-11-16 23:59:50,958 - tplexity.retriever.retriever_service - INFO - 🔍 [retriever_service] Начало поиска для запроса: Что такое банковский вклад?...
2025-11-16 23:59:50,959 - tplexity.llm_client.client - INFO - 🔄 [llm_client] Отправка запроса к LLM: model=QuantTrio/Qwen3-VL-30B-A3B-Instruct-AWQ, base_url=http://195.209.210.28:8000/v1
2025-11-16 23:59:51,327 - httpx - INFO - HTTP Request: POST http://195.209.210.28:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-16 23:59:51,334 - tplexity.llm_client.client - INFO - ✅ [llm_client] Ответ получен от LLM (model=QuantTrio/Qwen3-VL-30B-A3B-Instruct-AWQ)
2025-11-16 23:59:51,335 - tplexity.retriever.retriever_service - INFO - ✅ [retriever_service] Запрос переформулирован: 'Что такое банковский вклад?...' -> 'Что такое банковский вклад, его виды, условия и ос...'
Batches: 100%|██████████| 1/1 [00:00<00:00,  3.33it/s]
2025-11-16 23:59:52,443 - httpx - INFO - HTTP Request: POST https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.

Найдено результатов: 5
Doc ID: 6f4fadb3-df2d-4d34-bd88-f4bea9b1d8df, Score: 0.833
Text: 🏦 **В России появился банковский вклад со ставкой 35% годовых**

💳С 7 ноября банк «Дом.РФ» запустил ...
Metadata: {'doc_id': '1498653424_23333', 'message_id': 23333, 'channel_id': 1498653424, 'date': '2025-11-07T08:30:01+00:00', 'link': 'https://t.me/selfinvestor/23333'}

Doc ID: 598be75f-02a7-484f-a4b4-137b8468dabd, Score: 0.611
Text: 👀 **Участникам рынка следует придерживаться единого подхода к информированию клиентов**

Мы выявили ...
Metadata: {'doc_id': '1680634724_3070', 'message_id': 3070, 'channel_id': 1680634724, 'date': '2025-10-29T12:29:50+00:00', 'link': 'https://t.me/centralbank_russia/3070'}

Doc ID: 550c37ee-9c2f-41e4-8775-afa22c660644, Score: 0.533
Text: **🏛**** ЦБ: средняя ставка по вкладам в топ-10 банков упала ниже 15,4%**

Средняя максимальная ставк...
Metadata: {'doc_id': '1498653424_23301', 'message_id': 23301, 'channel_id': 1498653424, 'date': '2025-11-05T19:09:03+00:00', 'lin

In [7]:
results = await service.search("Как открыть счет?", top_k=5, top_n=3, use_rerank=False)
print(f"Найдено результатов: {len(results)}")
for doc_id, score, text, metadata in results:
    print(f"Doc ID: {doc_id}, Score: {score:.3f}")
    print(f"Text: {text[:100]}...")
    print(f"Metadata: {metadata}")
    print()

2025-11-16 23:59:54,082 - tplexity.retriever.retriever_service - INFO - 🔍 [retriever_service] Начало поиска для запроса: Как открыть счет?...
2025-11-16 23:59:54,083 - tplexity.llm_client.client - INFO - 🔄 [llm_client] Отправка запроса к LLM: model=QuantTrio/Qwen3-VL-30B-A3B-Instruct-AWQ, base_url=http://195.209.210.28:8000/v1
2025-11-16 23:59:54,277 - httpx - INFO - HTTP Request: POST http://195.209.210.28:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-16 23:59:54,278 - tplexity.llm_client.client - INFO - ✅ [llm_client] Ответ получен от LLM (model=QuantTrio/Qwen3-VL-30B-A3B-Instruct-AWQ)
2025-11-16 23:59:54,279 - tplexity.retriever.retriever_service - INFO - ✅ [retriever_service] Запрос переформулирован: 'Как открыть счет?...' -> 'Как открыть банковский счет: требования, документы...'
Batches: 100%|██████████| 1/1 [00:00<00:00, 50.29it/s]
2025-11-16 23:59:54,476 - httpx - INFO - HTTP Request: POST https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333

Найдено результатов: 3
Doc ID: 60decec1-d79e-49c9-835a-8cc0ea591b54, Score: 0.583
Text: **Банк** **России обновил рекомендации для предпринимателей, столкнувшихся с ограничениями операций ...
Metadata: {'doc_id': '1680634724_3039', 'message_id': 3039, 'channel_id': 1680634724, 'date': '2025-10-23T14:40:43+00:00', 'link': 'https://t.me/centralbank_russia/3039'}

Doc ID: faac9e29-c899-4fc1-9bc5-40d8f4f2b2f4, Score: 0.500
Text: 👀 **Банк России рекомендует банкам усилить контроль за процедурой идентификации для противодействия ...
Metadata: {'doc_id': '1680634724_3087', 'message_id': 3087, 'channel_id': 1680634724, 'date': '2025-11-01T08:48:26+00:00', 'link': 'https://t.me/centralbank_russia/3087'}

Doc ID: 24e13f6a-92f8-4e3d-9544-dd11669b267c, Score: 0.500
Text: Банк России направил на регистрацию в Минюст [документ,](http://www.cbr.ru/Crosscut/LawActs/File/100...
Metadata: {'doc_id': '1680634724_2825', 'message_id': 2825, 'channel_id': 1680634724, 'date': '2025-08-12T17:04:51+00:00', 'li

In [8]:
results = await service.search("кредит", top_k=3, top_n=2, use_rerank=False)
print(f"Найдено результатов (без rerank): {len(results)}")
for doc_id, score, text, metadata in results:
    print(f"Doc ID: {doc_id}, Score: {score:.3f}")
    print(f"Text: {text[:100]}...")
    print(f"Metadata: {metadata}")
    print()

2025-11-16 23:59:55,722 - tplexity.retriever.retriever_service - INFO - 🔍 [retriever_service] Начало поиска для запроса: кредит...
2025-11-16 23:59:55,723 - tplexity.llm_client.client - INFO - 🔄 [llm_client] Отправка запроса к LLM: model=QuantTrio/Qwen3-VL-30B-A3B-Instruct-AWQ, base_url=http://195.209.210.28:8000/v1
2025-11-16 23:59:56,464 - httpx - INFO - HTTP Request: POST http://195.209.210.28:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-16 23:59:56,465 - tplexity.llm_client.client - INFO - ✅ [llm_client] Ответ получен от LLM (model=QuantTrio/Qwen3-VL-30B-A3B-Instruct-AWQ)
2025-11-16 23:59:56,465 - tplexity.retriever.retriever_service - INFO - ✅ [retriever_service] Запрос переформулирован: 'кредит...' -> 'Кредит: виды, условия, требования, процессы, риски...'
Batches: 100%|██████████| 1/1 [00:00<00:00, 42.09it/s]
2025-11-16 23:59:56,647 - httpx - INFO - HTTP Request: POST https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections/documents

Найдено результатов (без rerank): 2
Doc ID: d4f3f98a-595b-47ea-85b2-754de9d2036e, Score: 0.583
Text: #БанковскийСектор**
В сентябре корпоративное кредитование немного замедлилось после существенного ро...
Metadata: {'doc_id': '1680634724_3023', 'message_id': 3023, 'channel_id': 1680634724, 'date': '2025-10-17T08:13:19+00:00', 'link': 'https://t.me/centralbank_russia/3023'}

Doc ID: c809d797-5bf5-4c94-8ba6-c3481a6c4f3a, Score: 0.500
Text: 🔧 **В сентябре-октябре денежно-кредитные условия оставались жесткими**
 
⏺Номинальные процентные ста...
Metadata: {'doc_id': '1680634724_3099', 'message_id': 3099, 'channel_id': 1680634724, 'date': '2025-11-11T17:14:14+00:00', 'link': 'https://t.me/centralbank_russia/3099'}



In [9]:
results = await service.search("ипотека", top_k=10, top_n=1, use_rerank=False)
if results:
    doc_id_to_delete = results[0][0]
    print(f"Найден документ для удаления: {doc_id_to_delete}")
    print(f"Text: {results[0][2][:100]}...")
else:
    doc_id_to_delete = None
    print("Не найдено документов")

2025-11-16 23:59:57,912 - tplexity.retriever.retriever_service - INFO - 🔍 [retriever_service] Начало поиска для запроса: ипотека...
2025-11-16 23:59:57,913 - tplexity.llm_client.client - INFO - 🔄 [llm_client] Отправка запроса к LLM: model=QuantTrio/Qwen3-VL-30B-A3B-Instruct-AWQ, base_url=http://195.209.210.28:8000/v1
2025-11-16 23:59:58,537 - httpx - INFO - HTTP Request: POST http://195.209.210.28:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-16 23:59:58,538 - tplexity.llm_client.client - INFO - ✅ [llm_client] Ответ получен от LLM (model=QuantTrio/Qwen3-VL-30B-A3B-Instruct-AWQ)
2025-11-16 23:59:58,539 - tplexity.retriever.retriever_service - INFO - ✅ [retriever_service] Запрос переформулирован: 'ипотека...' -> 'Ипотека: условия, ставки, требования к заемщикам, ...'
Batches: 100%|██████████| 1/1 [00:00<00:00, 49.27it/s]
2025-11-16 23:59:58,698 - httpx - INFO - HTTP Request: POST https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections/documen

Найден документ для удаления: 6540e714-fd2d-4d29-ac5b-f636b0db4a9b
Text: Председатель Банка России Эльвира Набиуллина — об ипотеке и депозитах.

__Смотрите видео с пресс-кон...


In [10]:
if doc_id_to_delete:
    await service.delete_documents([doc_id_to_delete])
    print(f"Удален документ: {doc_id_to_delete}")

    results = await service.search("ипотека", top_k=10, top_n=5, use_rerank=False)
    print(f"\nПоиск после удаления: найдено {len(results)} результатов")

2025-11-16 23:59:59,853 - tplexity.retriever.retriever_service - INFO - 🔄 [retriever_service] Удаление 1 документов
2025-11-16 23:59:59,854 - tplexity.retriever.vector_search - INFO - 🔄 [retriever][vector_search] Удаление 1 документов из коллекции documents
2025-11-16 23:59:59,901 - httpx - INFO - HTTP Request: POST https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections/documents/points/delete?wait=true "HTTP/1.1 200 OK"
2025-11-16 23:59:59,903 - tplexity.retriever.vector_search - INFO - ✅ [retriever][vector_search] Успешно удалено 1 документов из коллекции documents
2025-11-16 23:59:59,903 - tplexity.retriever.retriever_service - INFO - ✅ [retriever_service] Документы удалены из векторной базы
2025-11-16 23:59:59,903 - tplexity.retriever.retriever_service - INFO - 🔍 [retriever_service] Начало поиска для запроса: ипотека...
2025-11-16 23:59:59,904 - tplexity.llm_client.client - INFO - 🔄 [llm_client] Отправка запроса к LLM: model=QuantTrio/Qwen3

Удален документ: 6540e714-fd2d-4d29-ac5b-f636b0db4a9b


2025-11-17 00:00:00,553 - httpx - INFO - HTTP Request: POST http://195.209.210.28:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-17 00:00:00,554 - tplexity.llm_client.client - INFO - ✅ [llm_client] Ответ получен от LLM (model=QuantTrio/Qwen3-VL-30B-A3B-Instruct-AWQ)
2025-11-17 00:00:00,555 - tplexity.retriever.retriever_service - INFO - ✅ [retriever_service] Запрос переформулирован: 'ипотека...' -> 'Ипотека: условия, ставки, требования, документы, о...'
Batches: 100%|██████████| 1/1 [00:00<00:00, 42.53it/s]
2025-11-17 00:00:00,719 - httpx - INFO - HTTP Request: POST https://8b725c10-321f-428c-8fc8-00160f7dff00.eu-central-1-0.aws.cloud.qdrant.io:6333/collections/documents/points/query "HTTP/1.1 200 OK"
2025-11-17 00:00:00,761 - tplexity.retriever.retriever_service - INFO - ✅ [retriever_service] Гибридный поиск завершен, найдено результатов: 10
2025-11-17 00:00:00,762 - tplexity.retriever.retriever_service - INFO - ✅ [retriever_service] Поиск завершен, возвращено 5 результатов



Поиск после удаления: найдено 5 результатов


In [ ]:
await service.add_documents(["Дополнительный документ о банковских услугах"], metadatas=[{"source": "additional"}])
print("Добавлен дополнительный документ")

In [ ]:
results = await service.search("банковские услуги", top_k=10, top_n=5)
print(f"Найдено результатов после добавления: {len(results)}")
for doc_id, score, text, metadata in results:
    print(f"Doc ID: {doc_id}, Score: {score:.3f}")
    print(f"Text: {text[:80]}...")
    print(f"Metadata: {metadata}")
    print()

In [ ]:
await service.delete_all_documents()
print("Все документы удалены")

In [ ]:
results = await service.search("банковский вклад", top_k=10, top_n=5)
print(f"Найдено результатов после удаления всех: {len(results)}")
print(f"Results: {results}")

In [ ]:
# Тест получения всех документов через метод сервиса
all_documents = await service.get_all_documents()
print(f"Всего документов: {len(all_documents)}\n")
for i, (doc_id, text, metadata) in enumerate(all_documents, 1):
    print(f"Документ {i}:")
    print(f"  ID: {doc_id}")
    print(f"  Текст: {text}")
    if metadata:
        print(f"  Метаданные: {metadata}")
    print()

In [ ]:
# Тест получения одного документа по ID
# Получаем результаты поиска для получения ID
search_results_for_single = await service.search("банковский вклад", top_k=1, top_n=1, use_rerank=False)
if search_results_for_single:
    single_doc_id = [search_results_for_single[0][0]]  # Берем ID первого документа
    print(f"Получение одного документа по ID: {single_doc_id[0]}\n")

    single_doc = await service.get_documents(single_doc_id)
    if single_doc:
        doc_id, text, metadata = single_doc[0]
        print("Документ:")
        print(f"  ID: {doc_id}")
        print(f"  Текст: {text}")
        if metadata:
            print(f"  Метаданные: {metadata}")
    else:
        print("Документ не найден")
else:
    print("Нет документов для тестирования")

In [ ]:
# Тест гибридного поиска с MMR (Maximum Marginal Relevance)
# MMR добавлен в dense prefetch для увеличения разнообразия результатов
print("Тест гибридного поиска с MMR:\n")
results = await service.search("кредит банк", top_k=10, top_n=5, use_rerank=True)
print(f"Найдено результатов: {len(results)}")
print("\nРезультаты (с reranking и MMR):")
for i, (doc_id, score, text, metadata) in enumerate(results, 1):
    print(f"{i}. Doc ID: {doc_id}, Score: {score:.4f}")
    print(f"   Text: {text[:80]}...")
    if metadata:
        print(f"   Metadata: {metadata}")
    print()

In [ ]:
# Тест получения всех документов после добавления новых
print("Добавляем новые документы для теста...\n")
new_docs = [
    "Инвестиции - это вложение денежных средств с целью получения дохода в будущем.",
    "Страхование защищает от финансовых потерь в случае наступления страхового случая.",
]
new_metadatas = [{"type": "investment", "category": "finance"}, {"type": "insurance", "category": "finance"}]

await service.add_documents(new_docs, metadatas=new_metadatas)
print(f"Добавлено {len(new_docs)} новых документов\n")

# Получаем все документы
all_docs = await service.get_all_documents()
print(f"Всего документов в базе: {len(all_docs)}\n")
print("Последние добавленные документы:")
for doc_id, text, metadata in all_docs[-len(new_docs) :]:
    print(f"  ID: {doc_id}")
    print(f"  Text: {text}")
    print(f"  Metadata: {metadata}")
    print()

In [ ]:
# Тест получения документов по несуществующим ID
print("Тест получения документов по несуществующим ID:\n")
non_existent_ids = ["non-existent-id-1", "non-existent-id-2"]
retrieved = await service.get_documents(non_existent_ids)
print(f"Запрошено документов: {len(non_existent_ids)}")
print(f"Получено документов: {len(retrieved)}")
print("(Qdrant не возвращает документы с несуществующими ID, поэтому список будет пустым)")

In [ ]:
# Тест получения документов по ID
# Сначала получим ID документов из поиска
search_results = await service.search("банковский вклад", top_k=3, top_n=3, use_rerank=False)
if search_results:
    doc_ids = [doc_id for doc_id, _, _, _ in search_results]
    print(f"Получение документов по ID: {doc_ids}\n")

    # Получаем документы по ID
    retrieved_docs = await service.get_documents(doc_ids)
    print(f"Получено документов: {len(retrieved_docs)}\n")

    for i, (doc_id, text, metadata) in enumerate(retrieved_docs, 1):
        print(f"Документ {i}:")
        print(f"  ID: {doc_id}")
        print(f"  Текст: {text[:100]}...")
        if metadata:
            print(f"  Метаданные: {metadata}")
        print()
else:
    print("Нет документов для тестирования")